In [3]:
import pandas as pd
import numpy as np

In [4]:
# Skip the metadata rows and set the correct header
df = pd.read_excel("crop-area-and-production.xlsx", header=4)
df.columns.tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'crop-area-and-production.xlsx'

In [ ]:
df.columns = df.columns.str.strip().str.upper()
print(df.columns.tolist())

['LOCATION', 'YEAR', 'RICE_TA', 'RICE_TQ', 'WHT_TA', 'WHT_TQ', 'SORG_KA', 'SORG_KQ', 'SORG_RA', 'SORG_RQ', 'SORG_TA', 'SORG_TQ', 'PMLT_TA', 'PMLT_TQ', 'MAIZ_TA', 'MAIZ_TQ', 'FMLT_TA', 'FMLT_TQ', 'BRLY_TA', 'BRLY_TQ', 'CERL_TA', 'CERL_TQ', 'CPEA_TA', 'CPEA_TQ', 'PPEA_TA', 'PPEA_TQ', 'MPUL_TA', 'PULS_TA', 'PULS_TQ', 'GNUT_TA', 'GNUT_TQ', 'SESA_TA', 'SESA_TQ', 'RM_TA', 'RM_TQ', 'SAFF_TA', 'SAFF_TQ', 'CAST_TA', 'CAST_TQ', 'LINS_TA', 'LINS_TQ', 'SUNF_TA', 'SUNF_TQ', 'SOYA_TA', 'SOYA_TQ', 'OILS_TA', 'OILS_TQ', 'SCAN_TA', 'SGUR_TQ', 'COTN_TA', 'COTN_TQ', 'FRUT_TA', 'VEGT_TA']


In [ ]:
df.head()

,LOCATION,YEAR,RICE_TA,RICE_TQ,WHT_TA,WHT_TQ,SORG_KA,SORG_KQ,SORG_RA,SORG_RQ,...,SOYA_TA,SOYA_TQ,OILS_TA,OILS_TQ,SCAN_TA,SGUR_TQ,COTN_TA,COTN_TQ,FRUT_TA,VEGT_TA
0,All India,1950,30.81,20.58,9.75,6.46,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,10.73,5.16,1.71,57.05,5.88,0.5168,NaN,NaN
1,All India,1951,29.83,21.30,9.47,6.18,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,11.69,5.03,1.94,61.63,6.56,0.5576,NaN,NaN
2,All India,1952,29.97,22.90,9.83,7.50,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,11.18,4.73,1.73,51.00,6.36,0.5678,NaN,NaN
3,All India,1953,31.29,28.21,10.68,8.02,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,10.99,5.37,1.41,44.41,6.99,0.7021,NaN,NaN
4,All India,1954,30.77,25.22,11.26,9.04,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,12.52,6.40,1.62,58.74,7.55,0.7565,NaN,NaN


In [ ]:
df.replace(-1, np.nan, inplace=True)

In [ ]:
df["RICE_YIELD"] = df["RICE_TQ"] / df["RICE_TA"]
df["WHEAT_YIELD"] = df["WHT_TQ"] / df["WHT_TA"]

print(df[["YEAR", "RICE_YIELD", "WHEAT_YIELD"]].head())

   YEAR  RICE_YIELD  WHEAT_YIELD
0  1950    0.667965     0.662564
1  1951    0.714046     0.652587
2  1952    0.764097     0.762970
3  1953    0.901566     0.750936
4  1954    0.819630     0.802842


In [ ]:
def compatibility(crop1, crop2):
    # Rule 1: Cereal → Legume = Positive
    if crop1 in ["Rice","Wheat","Maize","Sorghum","Bajra","Barley"] and crop2 in ["Chickpea","Lentil","Pea","Mung Bean","Soybean"]:
        return 0.85, "Positive"
    
    # Rule 2: Same family crops = Negative
    if (crop1 in ["Rice","Wheat","Maize","Barley"] and crop2 in ["Rice","Wheat","Maize","Barley"]):
        return 0.6, "Negative"
    
    # Rule 3: Oilseed rotations = Neutral to Positive
    if crop2 in ["Mustard","Groundnut","Sunflower","Soybean"]:
        return 0.75, "Neutral"
    
    # Default
    return 0.7, "Neutral"

In [ ]:
rotation_rules = []

for crop1, season1 in season_map.items():
    for crop2, season2 in season_map.items():
        if crop1 != crop2:
            score, impact = compatibility(crop1, crop2)
            rotation_rules.append([crop1, crop2, season1, season2, score, impact])

# Convert to DataFrame
rules_df = pd.DataFrame(rotation_rules, columns=["Crop1","Crop2","Season1","Season2","Compatibility_Score","Soil_Impact"])

# Save to CSV
rules_df.to_csv("data/raw/rotation_rules.csv", index=False)

In [ ]:
print(rules_df.head(10))
print("Total rules generated:", len(rules_df))

  Crop1      Crop2 Season1 Season2  Compatibility_Score Soil_Impact
0  Rice      Maize  Kharif  Kharif                 0.60    Negative
1  Rice    Sorghum  Kharif  Kharif                 0.70     Neutral
2  Rice      Bajra  Kharif  Kharif                 0.70     Neutral
3  Rice    Soybean  Kharif  Kharif                 0.85    Positive
4  Rice     Cotton  Kharif  Kharif                 0.70     Neutral
5  Rice  Groundnut  Kharif  Kharif                 0.75     Neutral
6  Rice     Sesame  Kharif  Kharif                 0.70     Neutral
7  Rice      Wheat  Kharif    Rabi                 0.60    Negative
8  Rice     Barley  Kharif    Rabi                 0.60    Negative
9  Rice    Mustard  Kharif    Rabi                 0.75     Neutral
Total rules generated: 342


In [14]:
fertilizer_df=pd.read_csv("Crop and fertilizer dataset.csv")
icrisat_df=pd.read_csv("ICRISAT-District Level Data.csv")

In [15]:
fertilizer_df.head()

,District_Name,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer,Link
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo


In [16]:
icrisat_df.head()

,Dist Code,Year,State Code,State Name,Dist Name,RICE AREA (1000 ha),RICE PRODUCTION (1000 tons),RICE YIELD (Kg per ha),WHEAT AREA (1000 ha),WHEAT PRODUCTION (1000 tons),...,SUGARCANE YIELD (Kg per ha),COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),FRUITS AREA (1000 ha),VEGETABLES AREA (1000 ha),FRUITS AND VEGETABLES AREA (1000 ha),POTATOES AREA (1000 ha),ONION AREA (1000 ha),FODDER AREA (1000 ha)
0,95,2016,7,Maharashtra,Bombay,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96,2015,7,Maharashtra,Thane,133.90,306.5,2289.02,0.04,0.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96,2016,7,Maharashtra,Thane,132.70,375.3,2828.18,0.04,0.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96,2017,7,Maharashtra,Thane,132.68,246.3,1856.37,0.02,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,97,2015,7,Maharashtra,Raigad,115.60,339.9,2940.31,0.01,0.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
fertilizer_df.head()

,District_Name,Soil_color,Nitrogen,Phosphorus,Potassium,pH,Rainfall,Temperature,Crop,Fertilizer,Link
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo


In [18]:
icrisat_df.head()

,Dist Code,Year,State Code,State Name,Dist Name,RICE AREA (1000 ha),RICE PRODUCTION (1000 tons),RICE YIELD (Kg per ha),WHEAT AREA (1000 ha),WHEAT PRODUCTION (1000 tons),...,SUGARCANE YIELD (Kg per ha),COTTON AREA (1000 ha),COTTON PRODUCTION (1000 tons),COTTON YIELD (Kg per ha),FRUITS AREA (1000 ha),VEGETABLES AREA (1000 ha),FRUITS AND VEGETABLES AREA (1000 ha),POTATOES AREA (1000 ha),ONION AREA (1000 ha),FODDER AREA (1000 ha)
0,95,2016,7,Maharashtra,Bombay,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,96,2015,7,Maharashtra,Thane,133.90,306.5,2289.02,0.04,0.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,96,2016,7,Maharashtra,Thane,132.70,375.3,2828.18,0.04,0.10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,96,2017,7,Maharashtra,Thane,132.68,246.3,1856.37,0.02,0.02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,97,2015,7,Maharashtra,Raigad,115.60,339.9,2940.31,0.01,0.01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
district_to_region = {
    # Konkan
    
    "Thane": "Konkan", "Raigad": "Konkan", "Ratnagiri": "Konkan", "Sindhudurg": "Konkan",
    # Western Maharashtra
    "Pune": "Western Maharashtra", "Satara": "Western Maharashtra", "Sangli": "Western Maharashtra",
    "Kolhapur": "Western Maharashtra", "Nasik": "Western Maharashtra",
    # Marathwada
    "Aurangabad": "Marathwada", "Beed": "Marathwada", "Nanded": "Marathwada",
    "Parbhani": "Marathwada", "Jalna": "Marathwada", "Latur": "Marathwada",
    "Osmanabad": "Marathwada", "Hingoli": "Marathwada",
    # Vidarbha
    "Nagpur": "Vidarbha", "Wardha": "Vidarbha", "Amravati": "Vidarbha",
    "Akola": "Vidarbha", "Yavatmal": "Vidarbha", "Buldhana": "Vidarbha",
    "Chandrapur": "Vidarbha", "Gadchiroli": "Vidarbha"
}


In [26]:
fertilizer_df.rename(columns={
    "Nitrogen": "Optimal_N",
    "Phosphorus": "Optimal_P",
    "Potassium": "Optimal_K",
    "pH": "pH_Range"
}, inplace=True)


In [27]:
fertilizer_df.head()

,District_Name,Soil_color,Optimal_N,Optimal_P,Optimal_K,pH_Range,Rainfall,Temperature,Crop,Fertilizer,Link
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo


In [28]:
icrisat_df["Region"] = icrisat_df["Dist Name"].map(district_to_region)

In [29]:
crop_presence_cols = ["Dist Name", "Region", "RICE AREA (1000 ha)", "WHEAT AREA (1000 ha)",
                      "SORGHUM AREA (1000 ha)", "MAIZE AREA (1000 ha)", "CHICKPEA AREA (1000 ha)",
                      "PIGEONPEA AREA (1000 ha)", "GROUNDNUT AREA (1000 ha)", "SOYABEAN AREA (1000 ha)",
                      "COTTON AREA (1000 ha)"]


In [30]:
final_df = fertilizer_df.copy()

In [31]:
final_df

,District_Name,Soil_color,Optimal_N,Optimal_P,Optimal_K,pH_Range,Rainfall,Temperature,Crop,Fertilizer,Link
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo
...,...,...,...,...,...,...,...,...,...,...,...
4508,Pune,Black,130,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo
4509,Pune,Black,135,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo
4510,Pune,Black,140,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo
4511,Pune,Black,145,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo


In [32]:
final_df["Region"] = final_df["District_Name"].map(district_to_region)

In [33]:
final_df

,District_Name,Soil_color,Optimal_N,Optimal_P,Optimal_K,pH_Range,Rainfall,Temperature,Crop,Fertilizer,Link,Region
0,Kolhapur,Black,75,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
1,Kolhapur,Black,80,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
2,Kolhapur,Black,85,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
3,Kolhapur,Black,90,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
4,Kolhapur,Black,95,50,100,6.5,1000,20,Sugarcane,Urea,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
...,...,...,...,...,...,...,...,...,...,...,...,...
4508,Pune,Black,130,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
4509,Pune,Black,135,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
4510,Pune,Black,140,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo,Western Maharashtra
4511,Pune,Black,145,80,150,7.0,1400,30,Sugarcane,MOP,https://youtu.be/2t5Am0xLTOo,Western Maharashtra


In [35]:
final_df[["Crop", "Variety_Name", "Optimal_N", "Optimal_P", "Optimal_K", "pH_Range", "Region"]] \
    .to_csv("data/raw/regional_varieties.csv", index=False)


KeyError: "['Variety_Name'] not in index"